# 🚀 Stable Diffusion + Cloudflare Tunnel (FIXED v3.0)
## Google Colab Setup with Smart cloudflared Detection

### ✅ What this notebook does:
1. ✅ Verify GPU (Tesla T4/A100/L4)
2. ✅ Install cloudflared properly
3. ✅ Find cloudflared location (diagnostic)
4. ✅ Install Stable Diffusion WebUI
5. ✅ Launch WebUI + Tunnel
6. ✅ Get public HTTPS URL

## Cell 1: System Check & GPU Verification

In [ ]:
import subprocess
import os
import sys
import time
import psutil

print("\n" + "="*70)
print("[1/5] SYSTEM DIAGNOSTICS & GPU CHECK")
print("="*70)

# CPU Info
cpu_percent = psutil.cpu_percent(interval=1)
cpu_count = psutil.cpu_count()
print(f"\n📊 CPU:")
print(f"   • Cores: {cpu_count}")
print(f"   • Usage: {cpu_percent}%")

# Memory Info
mem = psutil.virtual_memory()
print(f"\n💾 RAM:")
print(f"   • Total: {mem.total / (1024**3):.1f} GB")
print(f"   • Available: {mem.available / (1024**3):.1f} GB")
print(f"   • Usage: {mem.percent}%")

# GPU Check
print(f"\n🎮 GPU Check:")
try:
    import torch
    if torch.cuda.is_available():
        print(f"   ✅ CUDA Available")
        print(f"   • Device: {torch.cuda.get_device_name(0)}")
        print(f"   • VRAM: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB")
    else:
        print(f"   ❌ CUDA NOT available")
        print(f"   → Go to Runtime → Change runtime type → Select GPU")
except Exception as e:
    print(f"   ⚠️ Error: {e}")

print("\n" + "="*70)
print("✅ System check complete")
print("="*70)

## Cell 2: Install & Verify Cloudflared (DIAGNOSTIC)

In [ ]:
import subprocess
import os
import shutil

print("\n" + "="*70)
print("[2/5] CLOUDFLARED INSTALLATION & DIAGNOSTIC")
print("="*70)

print("\n🔍 Step 1: Check if already installed...")
result = shutil.which('cloudflared')
if result:
    print(f"   ✅ Already found at: {result}")
else:
    print(f"   ❌ Not found in PATH")

print("\n📥 Step 2: Install via apt-get...")
result = subprocess.run(
    "sudo apt-get update && sudo apt-get install -y cloudflared",
    shell=True,
    capture_output=True,
    text=True,
    timeout=120
)

if result.returncode == 0:
    print(f"   ✅ Installation successful")
else:
    print(f"   ⚠️ Installation had issues")
    print(result.stderr[:200] if result.stderr else "(no error output)")

print("\n🔍 Step 3: Verify installation...")

# Method 1: which command
result = subprocess.run("which cloudflared", shell=True, capture_output=True, text=True)
if result.returncode == 0 and result.stdout.strip():
    cloudflared_path = result.stdout.strip()
    print(f"   ✅ Found via 'which': {cloudflared_path}")
else:
    print(f"   ❌ 'which' command failed")
    cloudflared_path = None

# Method 2: find command
if not cloudflared_path:
    print(f"\n   Trying 'find' command...")
    result = subprocess.run(
        "find /usr -name cloudflared -type f 2>/dev/null",
        shell=True,
        capture_output=True,
        text=True,
        timeout=10
    )
    if result.stdout.strip():
        cloudflared_path = result.stdout.strip().split('\n')[0]
        print(f"   ✅ Found via 'find': {cloudflared_path}")
    else:
        print(f"   ❌ 'find' command failed")

# Method 3: dpkg
if not cloudflared_path:
    print(f"\n   Checking via dpkg...")
    result = subprocess.run(
        "dpkg -L cloudflared 2>/dev/null | grep bin/cloudflared",
        shell=True,
        capture_output=True,
        text=True
    )
    if result.stdout.strip():
        cloudflared_path = result.stdout.strip().split('\n')[0]
        print(f"   ✅ Found via dpkg: {cloudflared_path}")
    else:
        print(f"   ❌ dpkg check failed")

# Test the binary
if cloudflared_path and os.path.exists(cloudflared_path):
    print(f"\n✅ CLOUDFLARED LOCATION CONFIRMED: {cloudflared_path}")
    result = subprocess.run(
        [cloudflared_path, "--version"],
        capture_output=True,
        text=True,
        timeout=5
    )
    if result.returncode == 0:
        version = result.stdout.strip().split('\n')[0]
        print(f"   Version: {version}")
    print(f"\n   💾 Saving path for next cell...")
    import json
    with open('/tmp/cloudflared_config.json', 'w') as f:
        json.dump({
            'cloudflared_path': cloudflared_path,
            'status': 'ready'
        }, f)
    print(f"   ✅ Saved")
else:
    print(f"\n❌ CRITICAL: Could not find cloudflared binary!")
    print(f"   Trying manual installation...")
    result = subprocess.run(
        "wget https://github.com/cloudflare/cloudflared/releases/download/2024.11.0/cloudflared-linux-amd64 -O /tmp/cloudflared && chmod +x /tmp/cloudflared && sudo cp /tmp/cloudflared /usr/local/bin/cloudflared",
        shell=True,
        capture_output=True,
        text=True,
        timeout=60
    )
    if result.returncode == 0:
        print(f"   ✅ Manual installation successful")
        cloudflared_path = "/usr/local/bin/cloudflared"
        with open('/tmp/cloudflared_config.json', 'w') as f:
            json.dump({
                'cloudflared_path': cloudflared_path,
                'status': 'ready'
            }, f)
    else:
        print(f"   ❌ Manual installation failed")
        print(result.stderr[:300])

print("\n" + "="*70)
print("✅ Cloudflared check complete")
print("="*70)

## Cell 3: Install WebUI & Dependencies

In [ ]:
import subprocess
import os
import time

print("\n" + "="*70)
print("[3/5] STABLE DIFFUSION WEBUI SETUP")
print("="*70)

webui_dir = "/root/stable-diffusion-webui"

print(f"\n📥 Cloning WebUI to {webui_dir}...")
if not os.path.exists(webui_dir):
    result = subprocess.run(
        ["git", "clone", "https://github.com/AUTOMATIC1111/stable-diffusion-webui", webui_dir],
        capture_output=True,
        timeout=300
    )
    if result.returncode == 0:
        print(f"   ✅ Cloned successfully")
    else:
        print(f"   ⚠️ Clone had issues, continuing anyway")
else:
    print(f"   ⏭️ Already exists")

os.chdir(webui_dir)

print(f"\n📦 Installing Python dependencies...")
commands = [
    ("pip install --upgrade pip setuptools wheel", "pip upgrade"),
    ("pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118", "PyTorch"),
    ("pip install transformers diffusers accelerate gradio omegaconf einops", "ML libraries"),
    ("pip install peft xformers requests Pillow", "Additional tools")
]

for i, (cmd, desc) in enumerate(commands, 1):
    print(f"   [{i}/{len(commands)}] Installing {desc}...")
    try:
        result = subprocess.run(
            cmd,
            shell=True,
            capture_output=True,
            timeout=180
        )
        if result.returncode == 0:
            print(f"        ✅ Done")
        else:
            print(f"        ⚠️ Some warnings (OK)")
    except subprocess.TimeoutExpired:
        print(f"        ⏱️ Timeout (continuing)")
    except Exception as e:
        print(f"        ⚠️ Error: {str(e)[:50]}")

print("\n" + "="*70)
print("✅ WebUI installation complete")
print("="*70)

## Cell 4: Launch WebUI & Tunnel (SMART VERSION)

In [ ]:
import subprocess
import time
import os
import re
import json

print("\n" + "="*70)
print("[4/5] LAUNCHING WEBUI & TUNNEL")
print("="*70)

# Kill old processes
print("\n🧹 Cleaning up old processes...")
subprocess.run("pkill -f 'python.*launch.py'", shell=True, stderr=subprocess.DEVNULL)
subprocess.run("pkill -f cloudflared", shell=True, stderr=subprocess.DEVNULL)
time.sleep(2)

# Launch WebUI
print("\n🚀 Starting WebUI...")
webui_dir = "/root/stable-diffusion-webui"
os.chdir(webui_dir)

webui_process = subprocess.Popen(
    ["python", "launch.py", "--api", "--cors-allow-origins=*", "--listen"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    cwd=webui_dir
)

print("   ⏳ Waiting 30 seconds for WebUI to initialize...")
time.sleep(30)
print("   ✅ WebUI running on http://localhost:7860")

# Load cloudflared path
print("\n🌐 Setting up Tunnel...")
cloudflared_path = None

try:
    with open('/tmp/cloudflared_config.json', 'r') as f:
        config = json.load(f)
        cloudflared_path = config.get('cloudflared_path')
        print(f"   ✅ Loaded cloudflared path from config: {cloudflared_path}")
except:
    print(f"   ⚠️ Config file not found, trying default paths...")
    import shutil
    cloudflared_path = shutil.which('cloudflared')
    if cloudflared_path:
        print(f"   ✅ Found via shutil.which: {cloudflared_path}")

if not cloudflared_path:
    print(f"   ❌ Could not find cloudflared!")
    print(f"      This is likely a Google Colab environment issue.")
    print(f"      Try running the apt-get install cell again.")
else:
    print(f"\n   🚀 Starting tunnel with: {cloudflared_path}")
    try:
        # Start tunnel process
        tunnel_process = subprocess.Popen(
            [cloudflared_path, "tunnel", "--url", "http://localhost:7860"],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
            universal_newlines=True
        )
        
        print("   ⏳ Waiting for tunnel URL (15 seconds)...\n")
        timeout = time.time() + 20
        tunnel_url = None
        
        while time.time() < timeout:
            try:
                line = tunnel_process.stdout.readline()
                if line:
                    print(f"      {line.rstrip()}")
                    # Search for URL
                    match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', line)
                    if match:
                        tunnel_url = match.group(0)
                        break
                time.sleep(0.1)
            except:
                pass
        
        if tunnel_url:
            print(f"\n" + "="*70)
            print(f"🎉 SUCCESS! TUNNEL URL OBTAINED")
            print(f"="*70)
            print(f"\n🌐 Public URL: {tunnel_url}")
            print(f"\n📋 NEXT STEPS:")
            print(f"   1. Copy the URL above")
            print(f"   2. Go to your GitHub Pages site")
            print(f"   3. Click ⚙️ Settings (top right)")
            print(f"   4. Paste URL in 'Cloudflared Tunnel URL' field")
            print(f"   5. Click 'Test Connection'")
            print(f"   6. Start generating! 🎨")
            print(f"\n" + "="*70)
            
            # Save for later use
            with open('/tmp/tunnel_url.txt', 'w') as f:
                f.write(tunnel_url)
        else:
            print(f"\n⚠️ No URL found in output")
            print(f"   But tunnel should be running on port 8000")
            print(f"   Try accessing: http://localhost:8000")
    
    except Exception as e:
        print(f"   ❌ Error launching tunnel: {e}")
        import traceback
        traceback.print_exc()

print(f"\n💡 Keep this notebook running!")
print(f"   Do NOT close this browser tab or this cell.")

## Cell 5: Test API Connection & Show Status

In [ ]:
import requests
import time

print("\n" + "="*70)
print("[5/5] TESTING API CONNECTION")
print("="*70)

api_url = "http://localhost:7860"

print(f"\n🔌 Testing WebUI API...")
for attempt in range(1, 4):
    try:
        response = requests.get(f"{api_url}/api/sd-models", timeout=5)
        if response.status_code == 200:
            print(f"   ✅ API responding (attempt {attempt})")
            data = response.json()
            if isinstance(data, list) and len(data) > 0:
                print(f"   ✅ Models loaded: {len(data)} checkpoint(s)")
                print(f"      • {data[0].get('model_name', 'Unknown')[:60]}")
            else:
                print(f"   ⚠️ Models still loading, try again in 30 seconds")
            break
    except requests.exceptions.ConnectionError:
        if attempt < 3:
            print(f"   ⏳ WebUI still initializing... (attempt {attempt}/3)")
            time.sleep(5)
        else:
            print(f"   ❌ Cannot reach WebUI after {attempt} attempts")
            print(f"      Try running this cell again in 60 seconds")
    except Exception as e:
        print(f"   ⚠️ Error: {str(e)[:100]}")
        break

print("\n" + "="*70)
print("🎉 SETUP COMPLETE!")
print("="*70)
print(f"\n✅ WebUI: http://localhost:7860")
print(f"✅ API: http://localhost:7860/api")
print(f"✅ Tunnel: Check Cell 4 output for public URL")
print(f"\n📌 DO NOT CLOSE THIS NOTEBOOK!")
print(f"   The tunnel and WebUI will stop if you do.")